In [2]:
!ls

Birding_Data_2016.csv       cleaned_birding_data.csv
Cleaning_birding_data.ipynb


In [3]:
import pandas as pd
import numpy as np

In [4]:
data = pd.read_csv("Birding_Data_2016.csv")

In [5]:
#data.head()

In [6]:
#data.tail()

We can see from the head of this data that there are repeated columns for each person to record their bird sightings. Unfortunately, while it's easy for entry, this data is hard to use for analysis in its current format. I'm going to change it so that there are only 4 data columns, and each bird sighting will be listed as a separate row. The column we will add will be "Birder", so that we can keep the data on who saw what, without having to repeat columns.

In [7]:
new_data = data.iloc[:, :3] #select only the first 3 columns and make it a new dataframe
new_data.head()

,Bright,Unnamed: 1,Unnamed: 2
0,Species,Date,Location
1,Pileated Woodpecker,1/1/2016,"CDA, ID"
2,Black-billed Magpie,1/1/2016,"CDA, ID"
3,Common Raven,1/1/2016,"CDA, ID"
4,Black-capped Chickadee,1/1/2016,"CDA, ID"


In [8]:
#create a new column to identify who saw the bird
new_data['Birder'] = new_data.columns[0]

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [9]:
new_data.head()

,Bright,Unnamed: 1,Unnamed: 2,Birder
0,Species,Date,Location,Bright
1,Pileated Woodpecker,1/1/2016,"CDA, ID",Bright
2,Black-billed Magpie,1/1/2016,"CDA, ID",Bright
3,Common Raven,1/1/2016,"CDA, ID",Bright
4,Black-capped Chickadee,1/1/2016,"CDA, ID",Bright


In [10]:
#the column names that we want to use are held in the first row, so we'll copy those
column_names=new_data[:1].values

new_data.columns = column_names[0] #because it's in a series, we assign the first item in column_names

#unfortunately, this renamed our birder column because of the way the data was previously structured
#we will rename it here because to append the other columns properly, the column names must be identical.
new_data.rename(columns={'Bright':'Birder'}, inplace=True)
new_data.head()

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3778: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,Species,Date,Location,Birder
0,Species,Date,Location,Bright
1,Pileated Woodpecker,1/1/2016,"CDA, ID",Bright
2,Black-billed Magpie,1/1/2016,"CDA, ID",Bright
3,Common Raven,1/1/2016,"CDA, ID",Bright
4,Black-capped Chickadee,1/1/2016,"CDA, ID",Bright


In [11]:
#we drop the first row, since that was our column names
new_data = new_data.drop(new_data.index[0])
new_data.head()

,Species,Date,Location,Birder
1,Pileated Woodpecker,1/1/2016,"CDA, ID",Bright
2,Black-billed Magpie,1/1/2016,"CDA, ID",Bright
3,Common Raven,1/1/2016,"CDA, ID",Bright
4,Black-capped Chickadee,1/1/2016,"CDA, ID",Bright
5,Wild Turkey,1/3/2016,"Hwy 95, ID",Bright


In [12]:
#we repeat the processing steps that we did with the first 3 rows with the next two sets of rows
#each set of 3 rows represents another person whose data we want to add to the end of the list

filios = data.iloc[:, 3:6]
other_guy = data.iloc[:, 6:9]
filios = filios.drop(filios.index[0])
other_guy = other_guy.drop(other_guy.index[0])
filios['Birder'] = filios.columns[0]
other_guy['Birder'] = other_guy.columns[0]

In [13]:
#we again do the same steps as before, to make sure these dataframes are also named properly
filios.columns = column_names[0] 
other_guy.columns = column_names[0] 

filios.rename(columns={'Bright':'Birder'}, inplace=True)
other_guy.rename(columns={'Bright':'Birder'}, inplace=True)
filios.head()

,Species,Date,Location,Birder
1,Northern Cardinal,1/1/2016,"Canton, MI",Filios
2,Blue Jay,1/1/2016,"Canton, MI",Filios
3,Mourning Dove,1/1/2016,"Canton, MI",Filios
4,Canada Goose,1/1/2016,"Canton, MI",Filios
5,White-crowned Sparrow,1/5/2016,"San Francisco, CA",Filios


In [14]:
#now, we add each of the dataframes together to get a new combined list
appended = new_data.append(filios)
appended = appended.append(other_guy)

In [15]:
#the format of the original data had empty spaces where Ben saw birds and the rest of us didn't
#it's safe to drop all of those instances
final_data = appended.dropna()

In [16]:
final_data.tail()
#we have the data all added together, but I see a small problem here
#all of the dates are not in the same format
#and the index column is messed up

,Species,Date,Location,Birder
37,House Sparrow,3Apr2016,"Mt. View Park Moscow, ID",Bowles
38,House Finch,Mar2016,WSU Campus,Bowles
39,American Wigeon,Mar2016,"Clarkston, WA",Bowles
40,American Coot,Mar2016,"Clarkston, WA",Bowles
41,House Wren,16Apr2016,"Moscow, ID",Bowles


In [17]:
#we can reset the index column numbers easily enough
reset = final_data.reset_index(drop=True)
reset.tail()

,Species,Date,Location,Birder
199,House Sparrow,3Apr2016,"Mt. View Park Moscow, ID",Bowles
200,House Finch,Mar2016,WSU Campus,Bowles
201,American Wigeon,Mar2016,"Clarkston, WA",Bowles
202,American Coot,Mar2016,"Clarkston, WA",Bowles
203,House Wren,16Apr2016,"Moscow, ID",Bowles


In [18]:
#I haven't ever changed something to a different date/time format before, let's try this pd.to_datetime function
#that I just learned about on stackexchange.
test_series = reset['Date']
test_series.shape
#print(test_series)
pd.to_datetime(test_series)

0     2016-01-01
1     2016-01-01
2     2016-01-01
3     2016-01-01
4     2016-01-03
5     2016-01-03
6     2016-01-04
7     2016-01-04
8     2016-01-04
9     2016-01-04
10    2016-01-04
11    2016-01-04
12    2016-01-04
13    2016-01-04
14    2016-01-04
15    2016-01-04
16    2016-01-04
17    2016-01-04
18    2016-01-04
19    2016-01-04
20    2016-01-04
21    2016-01-04
22    2016-01-04
23    2016-01-04
24    2016-01-04
25    2016-01-04
26    2016-01-05
27    2016-01-05
28    2016-01-05
29    2016-01-05
         ...    
174   2016-01-09
175   2016-01-09
176   2016-01-14
177   2016-01-23
178   2016-01-23
179   2016-01-27
180   2016-01-24
181   2016-01-31
182   2016-01-30
183   2016-01-29
184   2016-01-31
185   2016-02-03
186   2016-02-02
187   2016-02-03
188   2016-02-21
189   2016-02-20
190   2016-02-20
191   2016-02-19
192   2016-02-20
193   2016-03-10
194   2016-03-10
195   2016-03-02
196   2016-03-01
197   2016-04-03
198   2016-04-03
199   2016-04-03
200   2016-03-01
201   2016-03-

In [19]:
#looks like it worked pretty well! let's implement it on our data
reset['Clean_Dates'] = pd.to_datetime(reset['Date'])
reset.head()

,Species,Date,Location,Birder,Clean_Dates
0,Pileated Woodpecker,1/1/2016,"CDA, ID",Bright,2016-01-01
1,Black-billed Magpie,1/1/2016,"CDA, ID",Bright,2016-01-01
2,Common Raven,1/1/2016,"CDA, ID",Bright,2016-01-01
3,Black-capped Chickadee,1/1/2016,"CDA, ID",Bright,2016-01-01
4,Wild Turkey,1/3/2016,"Hwy 95, ID",Bright,2016-01-03


In [20]:
#let's drop out the date column and save it as a new object
clean_data = reset.drop('Date', axis = 1)

In [44]:
clean_data.head()
clean_data.columns
type(clean_data)

pandas.core.frame.DataFrame

In [22]:
# a fun project to try in the future: changing the location strings into geocode data
# then we can make a map of where all the birds were seen
# from pygeocoder import Geocoder

In [23]:
# print(clean_data['Location'][1])
# Geocoder.geocode(clean_data['Location'][0]).valid_address

In [317]:
#instead, let's go ahead and saved our cleaned up version as a new csv

clean_data.to_csv('cleaned_birding_data.csv', index=False)